In [1]:
!pip install influxdb-client pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.3/746.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 15.3 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()   # upload sample_data1.csv


Saving sensor_sample_1.csv to sensor_sample_1.csv


In [3]:
import pandas as pd

# Load CSV
csv_file = list(uploaded.keys())[0]   # should be sample_data1.csv
df = pd.read_csv(csv_file)

# --- CLEANING ---

# 1. Remove duplicates
df = df.drop_duplicates()

# 2. Handle missing values
df = df.fillna(method='ffill').fillna(method='bfill')

# 3. Clip out-of-range values (domain-based thresholds)
df['vibration'] = df['vibration'].clip(lower=0, upper=5)
df['temperature'] = df['temperature'].clip(lower=0, upper=120)
df['motor_load'] = df['motor_load'].clip(lower=0, upper=1)

# 4. Smooth signals (rolling mean)
df['vibration'] = df['vibration'].rolling(5, min_periods=1).mean()
df['temperature'] = df['temperature'].rolling(5, min_periods=1).mean()
df['motor_load'] = df['motor_load'].rolling(5, min_periods=1).mean()

print("✅ Cleaned data ready")
df.head()


✅ Cleaned data ready


/tmp/ipython-input-515787619.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


timestamp  vibration  temperature  motor_load  failure_soon
0  2025-01-01 00:00:00   0.004967    50.069657    0.490563             0
1  2025-01-01 00:00:01   0.004401    50.065661    0.505458             0
2  2025-01-01 00:00:02   0.006454    49.984673    0.502384             0
3  2025-01-01 00:00:03   0.011351    50.021234    0.502316             0
4  2025-01-01 00:00:04   0.013871    49.961384    0.504802             0

In [4]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import time

# ==== NEW PROJECT CONFIG ====
url = "https://us-east-1-1.aws.cloud2.influxdata.com/"   # region
token = "c7UeiQyNxW1tsxDZzAQA7lHLMMRZXQYkW9DLeNy69TviFduUYYGRDxqmFok1YxrvSulZpOB8rF_9x4IhB4Bnaw=="                            # paste new API key here
org = "factory-org"                                     # your org
bucket = "factory-data1"                                # new bucket
# ============================

# Connect
client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)

print("✅ Connected to InfluxDB Cloud (factory-data1)")


✅ Connected to InfluxDB Cloud (factory-data1)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier  # faster for demo
from sklearn.metrics import classification_report

# Features & Label
X = df[["vibration", "temperature", "motor_load"]]
y = df["failure_soon"]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train lightweight model
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("✅ Model trained")
print(classification_report(y_test, y_pred))


✅ Model trained
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       937
           1       1.00      0.51      0.67        63

    accuracy                           0.97      1000
   macro avg       0.98      0.75      0.83      1000
weighted avg       0.97      0.97      0.96      1000



In [ ]:
import time
from influxdb_client import Point

print("🚀 Streaming CLEANED data with predictions...")

while True:
    for _, row in df.iterrows():
        # ML prediction
        features = [[row["vibration"], row["temperature"], row["motor_load"]]]
        pred = int(model.predict(features)[0])

        point = (
            Point("machine")
            .tag("machine_id", "motor1")
            .field("vibration", float(row["vibration"]))
            .field("temperature", float(row["temperature"]))
            .field("motor_load", float(row["motor_load"]))
            .field("failure_soon", int(row["failure_soon"]))   # actual
            .field("predicted_failure", pred)                  # ML prediction
            .time(pd.Timestamp.now())
        )

        write_api.write(bucket=bucket, org=org, record=point)
        print(f"✅ Row sent | actual={int(row['failure_soon'])} | predicted={pred}")
        time.sleep(1)  # 1 row per sec


🚀 Streaming CLEANED data with predictions...
✅ Row sent | actual=0 | predicted=0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


✅ Row sent | actual=0 | predicted=0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


✅ Row sent | actual=0 | predicted=0


...

✅ Row sent | actual=0 | predicted=0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


✅ Row sent | actual=0 | predicted=0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


✅ Row sent | actual=0 | predicted=0
